*** ACCESS TO AND MANIPULATE A POSTGRESQL DATABASE FROM R ***

# 1. Using RPostgreSQL package

First load necessary package

In [ ]:
library(RPostgreSQL)

Load PostgreSQL driver

In [ ]:
drvv <- dbDriver("PostgreSQL")
drvv

Create a connection to a database using the driver

In [ ]:
con <- dbConnect(drvv,
                 dbname = "imdb",
                 host = "localhost",
                 port = 5432,
                 user = "postgres",
                 password = "bda505")

con

Get list of table names

In [ ]:
table_names <- dbGetQuery(con,
                          "SELECT table_name
                          FROM information_schema.tables 
                          WHERE table_type = 'BASE TABLE'
                          AND table_schema = 'public'
                          ORDER BY table_name
                          "
                          )

table_names

class(table_names)

Or more easily with:

In [ ]:
dbListTables(con)

Good! The query returns a data frame

Now we can loop through tables to automatize queries

Lets first get row counts of all tables, one by one

In [ ]:
# table_names is a data frame
# it is easier to work with and iterate through vectors
# so first extract the vector from the table_names
table_names_vec <- table_names[[1]]

# create a query template that we can change the parameters of
# each "%s" stands for a variable that we will manipulate with sprintf
query_text <- "SELECT count (*) FROM %s"

# create an empty vector, the length of the table names
# we will populate this vector with the row counts from tables
row_counts <- rep(NA, length(table_names_vec))

# update the names of the vector with table names
names(row_counts) <- table_names_vec

# for across table names indices (not the names themselves)
for (tbl_ind in seq_along(table_names_vec))
{
    # replace the %s pointer in query text with table name
    current_query <- sprintf(query_text, table_names_vec[tbl_ind])
    
    # run the query and get results
    table_name <- dbGetQuery(con, current_query)
    
    # assign the query result to vector
    row_counts[tbl_ind] <- table_name[[1]]
}
                  
return(row_counts)

In [ ]:
row_counts

In [ ]:
class(row_counts)

Now close the database connection

In [ ]:
dbDisconnect(con)

Now let's create a copy of the imdb database as imdb2 (so that imdb stays intact)

Open another connection to postgres database, so that we can play with imdb better

In [ ]:
con <- dbConnect(drvv,
                 dbname = "postgres",
                 host = "localhost",
                 port = 5432,
                 user = "postgres",
                 password = "bda505")

con

Kill all connections to imdb database (so that subsequent actions do not yield an error)

In [ ]:
dbGetQuery(con,
"
SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity 
WHERE pg_stat_activity.datname = 'imdb' AND pid <> pg_backend_pid();
"
                          )

Now create a copy of the imdb database as imdb2

In [ ]:
dbGetQuery(con,
"
CREATE DATABASE imdb2
WITH TEMPLATE imdb OWNER postgres
TABLESPACE = pg;
"
                          )

Close the connection

In [ ]:
dbDisconnect(con)

And open a new connection, this time to imdb2 database

In [ ]:
con <- dbConnect(drvv,
                 dbname = "imdb2",
                 host = "localhost",
                 port = 5432,
                 user = "postgres",
                 password = "bda505")

con

List tables:

In [ ]:
table_names_vec <- dbListTables(con)
table_names_vec
class(table_names_vec)

This yields a character vector, not a data frame

Get row counts

In [ ]:
# create a query template that we can change the parameters of
# each "%s" stands for a variable that we will manipulate with sprintf
query_text <- "SELECT count (*) FROM %s"

# create an empty vector, the length of the table names
# we will populate this vector with the row counts from tables
row_counts <- rep(NA, length(table_names_vec))

# update the names of the vector with table names
names(row_counts) <- table_names_vec

# for across table names indices (not the names themselves)
for (tbl_ind in seq_along(table_names_vec))
{
    # replace the %s pointer in query text with table name
    current_query <- sprintf(query_text, table_names_vec[tbl_ind])
    
    # run the query and get results
    table_name <- dbGetQuery(con, current_query)
    
    # assign the query result to vector
    row_counts[tbl_ind] <- table_name[[1]]
}
                  
return(row_counts)

Let's import a table as a data frame into R

In [ ]:
title_basics_df <- dbGetQuery(con, "SELECT * from title_basics")

Get the head, attributes and summary of the data frame

In [ ]:
head(title_basics_df)

In [ ]:
attributes(title_basics_df)

In [ ]:
summary(title_basics_df)

Now an easier way to read data into R

In [ ]:
title_basics_df_b <- dbReadTable(con, "title_basics")

See whether they are identical objects

In [ ]:
identical(title_basics_df, title_basics_df_b)

So, they are identical

What is the object size of each data frame?

In [ ]:
format(object.size(title_basics_df), units = "auto")

In [ ]:
format(object.size(title_basics_df_b), units = "auto")

They take too much space. Now let's get rid of one of them

In [ ]:
rm(title_basics_df_b)

New get the classes of all columns

In [ ]:
sapply(title_basics_df, class)

Now let's extract a subset of the data frame

Let's say, take rows in which 

name contains "Star Wars",

not an adult movie

title_type is a movie and

start year is between 1977 and 2016

In [ ]:
title_basics_df2 <- title_basics_df[
    grepl("(?i)star.*wars", title_basics_df$primarytitle) &
    !title_basics_df$isadult &
    title_basics_df$titletype == "movie" &
    title_basics_df$startyear >= 1977 &
    title_basics_df$startyear <= 2016,
]

In [ ]:
title_basics_df2

Did not like NA's in tconst, let's enhance it

In [ ]:
title_basics_df3 <- title_basics_df2[
    !is.na(title_basics_df2$tconst),
]

In [ ]:
title_basics_df3

Much better!

Now let's write this data frame into our imdb2 database as a new table

In [ ]:
dbWriteTable(con, "star_wars", value = title_basics_df3, append = TRUE, row.names = FALSE)

Check whether table is created:

In [ ]:
dbExistsTable(con, "star_wars")

And read from the database again

In [ ]:
star_wars <- dbReadTable(con, "star_wars")

star_wars

See, the only difference seems to be the rownames which we excluded on purpose

Apart from that, let's check whet they are both identical

In [ ]:
rownames(title_basics_df3) <- NULL
title_basics_df3

In [ ]:
identical(title_basics_df3, star_wars)

Yes they are identical!

Let's remove the table

In [ ]:
dbRemoveTable(con, "star_wars")

Check whether it still exists

In [ ]:
dbExistsTable(con, "star_wars")

Now let's create the table again

In [ ]:
dbWriteTable(con, "star_wars", value = title_basics_df3, append = TRUE, row.names = FALSE)

Check that it exists:

In [ ]:
dbExistsTable(con, "star_wars")

Get its rowcount:

In [ ]:
dbGetQuery(con, "SELECT count (*) FROM star_wars")

And repeat the data write statement, with append option

In [ ]:
dbWriteTable(con, "star_wars", value = title_basics_df3, append = TRUE, row.names = FALSE)

Get the rowcount again

In [ ]:
dbGetQuery(con, "SELECT count (*) FROM star_wars")

<a href="https://www.youtube.com/embed/tsXEToflqGs?start=10&autoplay=1">OW! I HAVE A BAD FEELING ABOUT THIS</a>

We did not define a primary key, so postgresql server allowed duplicates to be inserted in to the database

Now delete and recreate table

In [ ]:
dbRemoveTable(con, "star_wars")

In [ ]:
dbExistsTable(con, "star_wars")

In [ ]:
dbWriteTable(con, "star_wars", value = title_basics_df3, append = TRUE, row.names = FALSE)

In [ ]:
dbExistsTable(con, "star_wars")

And add a primary key constraint

In [ ]:
dbGetQuery(con,           
"
ALTER TABLE star_wars
ADD CONSTRAINT tconst_pk_07 
PRIMARY KEY (tconst);
")

Let's try to import the data again

In [ ]:
dbWriteTable(con, "star_wars", value = title_basics_df3, append = TRUE, row.names = FALSE)

See that:

*** could not Retrieve the result : ERROR:  duplicate key value violates unique constraint "tconst_pk_07 ***

Let's check the row count again:

In [ ]:
dbGetQuery(con, "SELECT count (*) FROM star_wars")

OK, we duplicates were really not allowed!

Let's close connection

In [ ]:
dbDisconnect(con)

And unload the driver

In [ ]:
dbUnloadDriver(drvv)

# 2. Using sqldf package

First let's load the package:

In [ ]:
library(sqldf)

And set the options:

In [ ]:
options(sqldf.RPostgreSQL.user ="postgres", 
  sqldf.RPostgreSQL.password ="bda505",
  sqldf.RPostgreSQL.dbname ="imdb2",
  sqldf.RPostgreSQL.host ="localhost", 
  sqldf.RPostgreSQL.port =5432)

Now let's run a query to get average runtimeminutes from title_basics and group them and order them by startyear  

In [ ]:
query = "SELECT avg(runtimeminutes) runtimeminutes, startyear 
            FROM title_basics
            GROUP BY startyear
            ORDER BY startyear;"

av_runtime <- sqldf(query)

In [ ]:
av_runtime

class(av_runtime)

It returns a data frame

Now subset years between 1940 and 2016

In [ ]:
av_runtime_subset <- av_runtime[av_runtime$startyear %in% 1940:2016,]

av_runtime_subset

Check whether any NA's exist

In [ ]:
which(is.na(av_runtime_subset[[1]]))

Let's plot the data

In [ ]:
runtime_ts <- ts(av_runtime_subset[[1]],
                 start = av_runtime_subset[1,2],
                 end = av_runtime_subset[nrow(av_runtime_subset),2])

runtime_ts

In [ ]:
plot(runtime_ts)

Well that data may have many different types of titles including shorts, etc

Let's first subset for movies

In [ ]:
query = "SELECT avg(runtimeminutes) runtimeminutes, startyear 
            FROM title_basics
            WHERE titletype = 'movie'
            GROUP BY startyear
            ORDER BY startyear;"

av_runtime <- sqldf(query)

In [ ]:
av_runtime

In [ ]:
av_runtime_subset <- av_runtime[av_runtime$startyear %in% 1940:2016,]

av_runtime_subset

In [ ]:
runtime_ts <- ts(av_runtime_subset[[1]],
                 start = av_runtime_subset[1,2],
                 end = av_runtime_subset[nrow(av_runtime_subset),2])

runtime_ts

In [ ]:
plot(runtime_ts)